In [ ]:
from perlin_noise import PerlinNoise
from common.utils import * 

import numpy as np

outp_dir = 'img/output_lattice_noise_images/'

figsize = (10, 8)

%matplotlib inline

In [ ]:
def graph_mesh(x_min, x_max, x_step):
    mesh = np.arange(x_min, x_max + 1, x_step)
    return mesh


def qubic_interp(x):
    return x * x * (3 - 2 * x)


def fifth_order_interp(x):
    return 6 * x ** 5 - 15 * x ** 4 + 10 * x ** 3


def gen_value_noise_1d(outp_noise_size, iterm_mesh_size, octaves_num, persistence, func=None):
    random_values = []
    harmonics = []
    amplitude = persistence

    for _ in range(octaves_num):
        rv = amplitude * np.random.rand(outp_noise_size // iterm_mesh_size + 1)
        octave = gen_single_octave_1d(rv, outp_noise_size, iterm_mesh_size, func=func)

        random_values.append(rv)
        harmonics.append(octave)
        iterm_mesh_size = iterm_mesh_size // 2
        amplitude *= persistence

    return harmonics, random_values


def gen_single_octave_1d(random_values, size, grid_size, func=None):
    num_cells_x = size // grid_size
    noise_map = np.zeros(size + 1)

    for x in range(size):
        cell_x = x // grid_size
        local_x = (x % grid_size) / grid_size
        left = random_values[cell_x]
        right = random_values[cell_x + 1]
        smooth_x = func(local_x) if func else local_x
        interp = left * (1 - smooth_x) + right * smooth_x
        noise_map[x] += interp
        
    noise_map[-1] = random_values[-1]
    return noise_map


def gen_single_octave_2d(random_values, outp_noise_shape, iterm_mesh_shape, func=None):
    h, w = outp_noise_shape
    gr_h, gr_w = iterm_mesh_shape
    num_cells_x = w // gr_w
    num_cells_y = h // gr_h
    noise_map = np.zeros((h + 1, w + 1))
    
    for y in range(h):
        for x in range(w):
            cell_x = x // gr_w
            cell_y = y // gr_h

            local_x = (x % gr_w) / gr_w
            local_y = (y % gr_h) / gr_h

            top_left = random_values[cell_y, cell_x]
            top_right = random_values[cell_y, cell_x + 1]
            bottom_left = random_values[cell_y + 1, cell_x]
            bottom_right = random_values[cell_y + 1, cell_x + 1]

            if func is None:
                smooth_x = local_x
                smooth_y = local_y
            else:
                smooth_x = func(local_x)
                smooth_y = func(local_y)

            interp_top = top_left * (1 - smooth_x) + top_right * smooth_x
            interp_bottom = bottom_left * (1 - smooth_x) + bottom_right * smooth_x
            noise_map[y, x] += interp_top * (1 - smooth_y) + interp_bottom * smooth_y          
            noise_map[-1, x] = bottom_left * (1 - smooth_x) + bottom_right * smooth_x
        noise_map[y, -1] = top_right * (1 - smooth_y) + bottom_right * smooth_y
    noise_map[-1, -1] = random_values[-1, -1]
    return noise_map


def gen_value_noise_2d(outp_noise_shape, iterm_mesh_shape, octave_nums, persistence, func=None):
    harmonics = []
    random_values = []
    h, w = outp_noise_shape
    gr_h, gr_w = iterm_mesh_shape
    amplitude = persistence  

    for _ in range(octave_nums):
        rv = amplitude * np.random.rand(h // gr_h + 1, w // gr_w + 1)
        octave = gen_single_octave_2d(rv, outp_noise_shape, (gr_h, gr_w), func=func)

        random_values.append(rv)
        harmonics.append(octave)
        gr_h, gr_w = gr_h // 2, gr_w // 2
        amplitude *= persistence

    # noise_map = np.sum(harmonics, axis=0)
    # noise_map = (noise_map - np.min(noise_map)) / (np.max(noise_map) - np.min(noise_map))
    return harmonics


def gen_img(x_size, y_size):
    xx = np.linspace(-x_size / 2, x_size / 2, x_size)
    yy = np.linspace(-y_size / 2, y_size / 2, y_size)
    whitenoise = np.random.normal(0, 1, (y_size, x_size))
    return whitenoise


def get_slice(arr, rows, cols):
    if len(rows) != len(cols):
        print('Row and column indexes arrays have different sizes')
        return

    arr_cols, arr_rows = arr.shape
    sliced_arr = np.zeros(len(rows))

    for index, (row, col) in enumerate(zip(rows, cols)):
        c_l, c_r = int(col), int(col) + 1
        r_t, r_b = int(row), int(row) + 1
        eta_col, eta_row = (col - c_l), (row - r_t)

        if c_r >= arr_cols:
            c_r = c_r - 1

        if r_b >= arr_rows:
            r_b = r_b - 1

        top = (1 - eta_row) * arr[r_t, c_l] + eta_row * arr[r_t, c_r]
        bottom = (1 - eta_row) * arr[r_b, c_l] + eta_row * arr[r_b, c_r]
        total = (1 - eta_col) * top + eta_col * bottom
        sliced_arr[index] = total
    return sliced_arr


def freq_pink_filter_1d_old(x_freq, factor=0.5, no_mean=False):
    x_freq = np.abs(x_freq)
    f = 1 / (1 + np.abs(x_freq))
    f = f ** factor
    if no_mean:
        f_mask = np.abs(x_freq) < 1
        f[f_mask] = 0
    return f


def freq_pink_filter_2d_old(x_freq, y_freq, factor=1, x_stretch=1, y_stretch=1, no_mean=False):
    f = freq_filter_2d(x_freq / x_stretch, y_freq / y_stretch)
    f = 1 / (1 + f)
    f = f ** factor
    f = np.where(f==1, 0, f) if no_mean else f
    return f

In [ ]:
# Noise parameters
size = 128
mesh_size_minor = 2
mesh_size_major = 32
x_mesh_major = graph_mesh(0, size, mesh_size_major // 2)
x_mesh_minor = graph_mesh(0, size, mesh_size_minor)

# Graphs X-axis mesh
x_mesh_major = graph_mesh(0, size, mesh_size_major)
x_mesh_minor = graph_mesh(0, size, mesh_size_minor)

# Random gaussian noise for single octave
random_values = np.random.rand(size // mesh_size_major + 1)

# Different interpolation noises 
lin_noise = gen_single_octave_1d(random_values, size, mesh_size_major)
qubic_noise = gen_single_octave_1d(random_values, size, mesh_size_major, func=qubic_interp)
penta_noise = gen_single_octave_1d(random_values, size, mesh_size_major, func=fifth_order_interp)

# Plotting
ax = plot_graphs(lin_noise, qubic_noise, penta_noise)
ax.scatter(x_mesh_major, random_values, marker='.')

# Graph layout
ax.set_xticks(x_mesh_major)
ax.set_xticks(x_mesh_minor, minor=True)
ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
# plt.savefig(outp_dir + 'types_of_interp.png')
plt.show()

In [ ]:
# Mesh 
mesh_size_minor = 2
mesh_size_major = period = 32
x_mesh_major = graph_mesh(0, size, mesh_size_major // 2)
x_mesh_minor = graph_mesh(0, size, mesh_size_minor)

# Octave approach parameters
octaves = 3
pers = 0.707
amps = [pers ** (i + 1) for i in range(octaves)]
harmonics = []
random_values = []

# Graph setup
nrows, ncols = 2, 2
fig = plt.figure(figsize=figsize)


for i in range(octaves):
    rv = np.random.rand(size // period + 1)
    x_scatter = graph_mesh(0, size, period)
    octave = gen_single_octave_1d(rv, size, period, func=qubic_interp)
    
    random_values.append(rv)
    harmonics.append(octave)
    period = period // 2
    
    ax = fig.add_subplot(nrows, ncols, i + 1)
    ax.plot(octave)   
    ax.scatter(x_scatter, random_values[i], marker='.')

    ax.set_xticks(x_mesh_major)
    ax.set_xticks(x_mesh_minor, minor=True)

    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    ax.set_xbound(-1, size + 1)
    ax.set_ybound(0)

    plt.xlabel('X')
    plt.ylabel('Y')
    
harmonics = np.sum(harmonics, axis=0)
harmonics = normalize(harmonics) if np.max(harmonics) > 1 else harmonics      
scatter_vals = harmonics[x_mesh_major]  

ax = fig.add_subplot(nrows, ncols, i + 2)
ax.plot(harmonics)
ax.scatter(x_mesh_major, scatter_vals, marker='.')

ax.set_xticks(x_mesh_major)
ax.set_xticks(x_mesh_minor, minor=True)

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
plt.subplots_adjust(wspace=0.25, hspace=0.2)
# plt.savefig(outp_dir + 'cubic_octave_sum.png')
plt.show()

In [ ]:
# Mesh 
mesh_size_minor = 2
mesh_size_major = period = 32
x_mesh_major = graph_mesh(0, size, mesh_size_major // 2)
x_mesh_minor = graph_mesh(0, size, mesh_size_minor)

# Octave approach parameters
octaves = 3
pers = 0.707
amps = [pers ** (i + 1) for i in range(octaves)]
harmonics = []
fig = plt.figure(figsize=figsize)

for i in range(octaves):
    x_scatter = graph_mesh(0, size, period)
    octave = gen_single_octave_1d(random_values[i], size, period, func=fifth_order_interp)
    
    harmonics.append(octave)
    period = period // 2
    
    ax = fig.add_subplot(nrows, ncols, i + 1)
    ax.plot(octave)   
    ax.scatter(x_scatter, random_values[i], marker='.')

    ax.set_xticks(x_mesh_major)
    ax.set_xticks(x_mesh_minor, minor=True)

    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    ax.set_xbound(-1, size + 1)
    ax.set_ybound(0)

    plt.xlabel('X')
    plt.ylabel('Y')
    
harmonics = np.sum(harmonics, axis=0)
harmonics = normalize(harmonics) if np.max(harmonics) > 1 else harmonics      
scatter_vals = harmonics[x_mesh_major]  

ax = fig.add_subplot(nrows, ncols, i + 2)
ax.plot(harmonics)
ax.scatter(x_mesh_major, scatter_vals, marker='.')

ax.set_xticks(x_mesh_major)
ax.set_xticks(x_mesh_minor, minor=True)

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
plt.subplots_adjust(wspace=0.25, hspace=0.2)
# plt.savefig(outp_dir + 'fifth_order_octave_sum.png')
plt.show()

### White-to-pink kernel test

In [ ]:
t_size = 10000
freq = freq_numbers_1d(t_size)[t_size // 2:]

f1 = np.array([1 / (x ** 0.5) for x in range(1, t_size // 2)])
f2 = freq_pink_filter_1d_old(freq, factor=0.5)
f3 = freq_pink_filter_1d(freq, factor=0.5)

f1_psd = f1 ** 2
f2_psd = f2 ** 2
f3_psd = f3 ** 2

f1_db = 10 * np.log10(f1_psd)
f2_db = 10 * np.log10(f2_psd)
f3_db = 10 * np.log10(f3_psd)

plt.semilogx(freq[1:], f1_db)
plt.semilogx(freq, f2_db)
plt.semilogx(freq, f3_db)

plt.xlabel('k')
plt.ylabel('dB')
plt.grid()

In [ ]:
plt.plot(freq[1:], np.log10(f1))
plt.plot(freq, np.log10(f2))
plt.plot(freq, np.log10(f3))
plt.grid()

In [ ]:
N = 1000000
freq_filter = freq_pink_filter_1d([i for i in range(N)])

signal = np.random.randn(N)
signal -= np.mean(signal)

pink_noise_ft = find_ft_1d(signal) * freq_filter
pink_noise = find_ift_1d(pink_noise_ft)
ach = np.abs(pink_noise_ft)

plt.loglog(ach / np.max(ach))
plt.loglog(freq_filter)
plt.grid()

In [ ]:
# Parameters
num_samples = 1024
sampling_rate = 1000  # Hz

# Generate pink noise using the Voss-McCartney algorithm
pink_noise = np.random.randn(num_samples)
pink_noise = np.cumsum(pink_noise)
pink_noise -= np.mean(pink_noise)  # Center the signal

# Calculate the power spectral density
pink_freqs = np.fft.fftfreq(num_samples, d=1/sampling_rate)
pink_spectrum = np.abs(np.fft.fft(pink_noise)) ** 2

# Calculate the 1/f spectrum
frequencies = np.fft.fftfreq(num_samples, d=1/sampling_rate)
frequencies = frequencies[1:num_samples//2]
one_over_f_spectrum = 1 / frequencies

# Spectrum factor
power_factor = (np.sum(pink_spectrum ** 2) / np.sum(one_over_f_spectrum ** 2)) ** 0.5

# Create the plot
plt.figure(figsize=figsize)
plt.loglog(pink_freqs[:num_samples//2], pink_spectrum[:num_samples//2], label='Pink Noise')
plt.loglog(frequencies, one_over_f_spectrum, label='1/f Noise', linestyle='dashed')
plt.loglog(frequencies, power_factor * one_over_f_spectrum, label='1/f Noise Normalized', linestyle='dashed')

plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.title('Pink Noise Spectrum vs. 1/f Noise Spectrum')
plt.legend()
plt.grid(True)
plt.show()

### Spectrum

In [ ]:
qubic_noise -= np.mean(qubic_noise)
qubic_noise_ft = find_ft_1d(qubic_noise)
qubic_noise_mang = np.abs(qubic_noise_ft)
noise_size = len(qubic_noise)
freq = freq_numbers_1d(noise_size)[noise_size // 2:]
f_curve = freq_pink_filter_1d(freq)

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(1, 2, 1)
ax.loglog(freq, qubic_noise_mang[size // 2:])
ax.loglog(freq, f_curve)
ax.grid(which='both')
ax.set_ybound(10 ** -3, 100)
plt.xlabel('k')
plt.ylabel('АЧХ')


octaves = harmonics - np.mean(harmonics)
if np.max(octaves) > 1:
    octaves = normalize(octaves)
octaves_ft = np.abs(find_ft_1d(octaves)) 

ax = fig.add_subplot(1, 2, 2)
ax.loglog(octaves_ft[noise_size // 2:])
ax.loglog(f_curve)
ax.grid(which='both')
ax.set_ybound(10 ** -3, 100)
plt.xlabel('k')
plt.ylabel('АЧХ')
plt.subplots_adjust(wspace=0.25, hspace=0.2)
# plt.savefig(outp_dir + 'noise_spectrum_total.png')

### 2D case

In [ ]:
shape = (128, 128)
grid_size = (32, 32)
pers = 0.707
oct_num = 6

noise_2d = gen_value_noise_2d(shape, grid_size, oct_num, pers, func=qubic_interp)

fig = plt.figure(figsize=figsize)
ax1 = fig.add_subplot(1, 2, 1)
im1 = ax1.imshow(np.sum(noise_2d, axis=0), cmap='gray')

In [ ]:
grid_size = (16, 16)

noise_2d = gen_value_noise_2d(shape, grid_size, 1, pers, func=qubic_interp)
noise_2d = np.sum(noise_2d, axis=0)

noise_2d_oct = gen_value_noise_2d(shape, grid_size, 5, pers, func=qubic_interp)
noise_2d_oct = np.sum(noise_2d_oct, axis=0)

fig = plt.figure(figsize=(10, 4))
ax1 = fig.add_subplot(1, 2, 1)
im1 = ax1.imshow(noise_2d, cmap='gray', vmin=0, vmax=1)
# fig.colorbar(im, ax=ax1)
plt.xlabel('а')

ax = fig.add_subplot(1, 2, 2)
im = ax.imshow(noise_2d_oct, cmap='gray', vmin=0, vmax=2)
plt.xlabel('б')

cax = fig.add_axes([0.9, 0.11, 0.02, 0.77])
cbar = plt.colorbar(im, cax=cax)

plt.subplots_adjust(wspace=0)
# plt.savefig(outp_dir + 'value_noise_2d_case.png')
plt.show()

### Spectrum

In [ ]:
noise_2d_sp = noise_2d - np.mean(noise_2d)
noise_2d_sp = find_ft_2d(noise_2d_sp)

noise_2d_oct_sp = noise_2d_oct - np.mean(noise_2d_oct)
noise_2d_oct_sp = find_ft_2d(noise_2d_oct_sp)

plt.imshow(np.log(np.abs(noise_2d_sp)), cmap='gray')
plt.colorbar()
plt.show()

plt.imshow(np.log(np.abs(noise_2d_oct_sp)), cmap='gray')
plt.colorbar()
plt.show()

### Perlin Noise

In [ ]:
noise = PerlinNoise(octaves=3)
noise1 = PerlinNoise(octaves=6)
noise2 = PerlinNoise(octaves=12)
noise3 = PerlinNoise(octaves=24)
noise4 = PerlinNoise(octaves=48)

xpix, ypix = 128, 128

pic_single_oct = [[noise([i/xpix, j/ypix]) for j in range(xpix)] for i in range(ypix)]
pic_single_oct = normalize(pic_single_oct)
pic_oct = []

for i in range(xpix):
    row = []
    for j in range(ypix):
        noise_val = 0.707 * noise1([i/xpix, j/ypix])
        noise_val += 0.707 ** 2 * noise2([i/xpix, j/ypix])
        noise_val += 0.707 ** 3 * noise3([i/xpix, j/ypix])
        noise_val += 0.707 ** 4 * noise4([i/xpix, j/ypix])
        row.append(noise_val)
    pic_oct.append(row)
pic_oct = normalize(pic_oct)
    

fig = plt.figure(figsize=(10, 4))
ax1 = fig.add_subplot(1, 2, 1)
im1 = ax1.imshow(pic_single_oct, cmap='gray', vmin=0, vmax=1)
plt.xlabel('а')

ax = fig.add_subplot(1, 2, 2)
im = ax.imshow(pic_oct, cmap='gray', vmin=0, vmax=1)
plt.xlabel('б')

cax = fig.add_axes([0.9, 0.11, 0.02, 0.77])  # Adjust these values as needed
cbar = plt.colorbar(im, cax=cax)

plt.subplots_adjust(wspace=0)
# plt.savefig(outp_dir + 'perlin_noise_2d_case.png')
plt.show()